1. dividend growth twice the rate of inflation over last 3 years
2. flat or declinging payout ratio over last 3 years
    - Payout Ratio = (Dividends per Share (DPS) / Earnings per Share (EPS)) x 100
    - Payout Ratio = (Total Dividends Paid / Net Income) x 100
3. 3 year average payout ratio less than 60%
4. debt to equity ratio under 1.5
    - Debt/Equity Ratio = Total Liabilities / Shareholders' Equity
5. maintain net margins over 7% 
    - Net Margin = Net Income / Total Revenues
6. ROIC level exceeds 20%
    - Net Operating Profit After Tax (NOPAT)/Invested Capital = ROIC
7. decreasing diluted share count

In [1]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

from pandas.io.json import json_normalize
import urllib.request as req
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
from helpers import *

import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os


from scipy.stats import norm as norm
from yahoo_query import *

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [2]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

stock_list = pd.read_csv('companylist.csv')

start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    
    def create_dgi(balanceSheet, incomeStatement, cashFlowStatement):
        dgi_data = balanceSheet[['totalLiab',
                                 'totalStockholderEquity',
                                 'longTermInvestments',
                                 'shortTermInvestments']]
        
        dgi_data = dgi_data.join(incomeStatement[['netIncome',
                                                  'operatingIncome',
                                                  'totalRevenue']])
        
        dgi_data = dgi_data.join(cashFlowStatement[['dividendsPaid',
                                                    'investments']])
        dgi_data['dividendsPaid'] = -dgi_data['dividendsPaid']

        dgi_data['payoutRatio'] = dgi_data.dividendsPaid/dgi_data.netIncome
        dgi_data['debtEquityRatio'] = dgi_data.totalLiab/dgi_data.totalStockholderEquity
        dgi_data['netMargin'] = dgi_data.netIncome/dgi_data.totalRevenue
        dgi_data['roic'] = dgi_data.operatingIncome/(dgi_data.longTermInvestments + dgi_data.shortTermInvestments)
        
        dgi_data = dgi_data[['payoutRatio','debtEquityRatio','netMargin','roic','dividendsPaid']]
        dgi_data['Underyling'] = ticker
        return dgi_data
    
    dgi_annual = create_dgi(yahoo_data.balanceSheetAnnual, yahoo_data.incomeStatementAnnual, 
                            yahoo_data.cashFlowStatementAnnual)

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')
    
    return (dgi_annual, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10, lookback_years = 3):

    earnings_lst = []
    keyStats_lst = []
    dgi_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            keyStats_lst.append(curr_keyStats)
            
            dgi_df = pd.DataFrame(index = [ticker])
            dgi_df['payoutChange'] = curr_earnings.payoutRatio.sort_index().pct_change().tail(lookback_years).mean()
            dgi_df['divGrowth'] = curr_earnings.dividendsPaid.sort_index().pct_change().tail(lookback_years).mean()
            dgi_df['payoutAvg'] = curr_earnings.payoutRatio.sort_index().tail(lookback_years).mean()
            dgi_df['debtEquityAvg'] = curr_earnings.debtEquityRatio.sort_index().tail(lookback_years).mean()
            dgi_df['netMarginAvg'] = curr_earnings.netMargin.sort_index().tail(lookback_years).mean()
            dgi_df['roicAvg'] = curr_earnings.roic.sort_index().tail(lookback_years).mean()

            dgi_lst.append(dgi_df)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    keyStats_lst.append(curr_keyStats)
                    
                    dgi_df = pd.DataFrame(index = [ticker])
                    dgi_df['payoutChange'] = curr_earnings.payoutRatio.sort_index().pct_change().tail(lookback_years).mean()
                    dgi_df['divGrowth'] = curr_earnings.dividendsPaid.sort_index().pct_change().tail(lookback_years).mean()
                    dgi_df['payoutAvg'] = curr_earnings.payoutRatio.sort_index().tail(lookback_years).mean()
                    dgi_df['debtEquityAvg'] = curr_earnings.debtEquityRatio.sort_index().tail(lookback_years).mean()
                    dgi_df['netMarginAvg'] = curr_earnings.netMargin.sort_index().tail(lookback_years).mean()
                    dgi_df['roicAvg'] = curr_earnings.roic.sort_index().tail(lookback_years).mean()

                    dgi_lst.append(dgi_df)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))
    
    try:
        earnings_df = pd.concat(earnings_lst, axis = 0)
        earnings_df = earnings_df.reset_index()[earnings_df.columns]
        keyStats_df = pd.concat(keyStats_lst, axis = 0)
        dgi_df = pd.concat(dgi_lst, axis = 0)

        return earnings_df, keyStats_df, dgi_df, failed_list
    except:
        return failed_list


In [3]:
earnings_df, keyStats_df, dgi_df, failed_list = download_yahoo_data(stock_list['Symbol'].tolist(),
                                                                    10, 3)

earnings_df.fillna(0).to_csv('dgi_raw.csv')
keyStats_df.to_csv('keystats.csv')
dgi_df.fillna(0).to_csv('dgi_scores.csv')

Failed: 14D
0.01% Completed
1 total failures
Failed: 1AD
0.02% Completed
2 total failures
Failed: 1AG
0.04% Completed
3 total failures
Failed: 1AL
0.05% Completed
4 total failures
Failed: 1PG
0.06% Completed
5 total failures
Failed: 1ST
0.07% Completed
6 total failures
Failed: 3DP
0.08% Completed
7 total failures
Failed: 3PL
0.10% Completed
8 total failures
Failed: 4CE
0.11% Completed
9 total failures
Failed: 4DS
0.12% Completed
10 total failures
Failed: 5GN
0.13% Completed
11 total failures
Failed: 88E
0.15% Completed
12 total failures
Failed: 8CO
0.16% Completed
13 total failures
Failed: 8EC
0.17% Completed
14 total failures
Failed: 8IH
0.18% Completed
15 total failures
Failed: 9SP
0.19% Completed
16 total failures
Failed: A
0.21% Completed
17 total failures
Failed: A1C
0.22% Completed
18 total failures
Failed: A2M
0.23% Completed
19 total failures
Failed: A3D
0.24% Completed
20 total failures
Failed: AA
0.25% Completed
21 total failures
Failed: AABA
0.27% Completed
22 total failures

Failed: AFA
2.13% Completed
160 total failures
Failed: AFB
2.14% Completed
161 total failures
Failed: AFC
2.15% Completed
162 total failures
Failed: AFG
2.17% Completed
163 total failures
Failed: AFGE
2.18% Completed
164 total failures
Failed: AFGH
2.19% Completed
165 total failures
Failed: AFH
2.20% Completed
166 total failures
Failed: AFHBL
2.21% Completed
167 total failures
Failed: AFI
2.23% Completed
168 total failures
Failed: AFIN
2.24% Completed
169 total failures
Accepted: AFL
2.25% Completed
169 total failures
Failed: AFMD
2.26% Completed
170 total failures
Failed: AFP
2.27% Completed
171 total failures
Failed: AFR
2.29% Completed
172 total failures
Accepted: AFSI
2.30% Completed
172 total failures
Failed: AFSI^A
2.31% Completed
173 total failures
Failed: AFSI^B
2.32% Completed
174 total failures
Failed: AFSI^C
2.33% Completed
175 total failures
Failed: AFSI^D
2.35% Completed
176 total failures
Failed: AFSI^E
2.36% Completed
177 total failures
Failed: AFSI^F
2.37% Completed
178

Failed: ALL^G
4.20% Completed
319 total failures
Failed: ALLE
4.21% Completed
320 total failures
Failed: ALLK
4.22% Completed
321 total failures
Failed: ALLT
4.23% Completed
322 total failures
Failed: ALLY
4.25% Completed
323 total failures
Failed: ALLY^A
4.26% Completed
324 total failures
Failed: ALNA
4.27% Completed
325 total failures
Failed: ALNY
4.28% Completed
326 total failures
Failed: ALOT
4.29% Completed
327 total failures
Failed: ALP^Q
4.31% Completed
328 total failures
Failed: ALPN
4.32% Completed
329 total failures
Failed: ALQ
4.33% Completed
330 total failures
Failed: ALQA
4.34% Completed
331 total failures
Failed: ALRM
4.35% Completed
332 total failures
Failed: ALRN
4.37% Completed
333 total failures
Failed: ALSK
4.38% Completed
334 total failures
Failed: ALSN
4.39% Completed
335 total failures
Failed: ALT
4.40% Completed
336 total failures
Failed: ALTR
4.42% Completed
337 total failures
Failed: ALTY
4.43% Completed
338 total failures
Failed: ALU
4.44% Completed
339 total 

Failed: APVO
6.28% Completed
479 total failures
Failed: APW
6.29% Completed
480 total failures
Failed: APWC
6.30% Completed
481 total failures
Failed: APX
6.31% Completed
482 total failures
Failed: APY
6.33% Completed
483 total failures
Failed: APZ
6.34% Completed
484 total failures
Failed: AQ
6.35% Completed
485 total failures
Failed: AQB
6.36% Completed
486 total failures
Failed: AQC
6.37% Completed
487 total failures
Failed: AQD
6.39% Completed
488 total failures
Failed: AQG
6.40% Completed
489 total failures
Failed: AQH
6.41% Completed
490 total failures
Failed: AQI
6.42% Completed
491 total failures
Failed: AQJ
6.44% Completed
492 total failures
Failed: AQM
6.45% Completed
493 total failures
Failed: AQMS
6.46% Completed
494 total failures
Failed: AQN
6.47% Completed
495 total failures
Failed: AQS
6.48% Completed
496 total failures
Failed: AQST
6.50% Completed
497 total failures
Failed: AQUA
6.51% Completed
498 total failures
Failed: AQX
6.52% Completed
499 total failures
Failed: A

Failed: AUDC
8.36% Completed
641 total failures
Failed: AUH
8.37% Completed
642 total failures
Failed: AUI
8.38% Completed
643 total failures
Failed: AUL
8.39% Completed
644 total failures
Accepted: AUO
8.41% Completed
644 total failures
Failed: AUP
8.42% Completed
645 total failures
Failed: AUPH
8.43% Completed
646 total failures
Failed: AUQ
8.44% Completed
647 total failures
Failed: AUR
8.46% Completed
648 total failures
Failed: AUTL
8.47% Completed
649 total failures
Failed: AUTO
8.48% Completed
650 total failures
Accepted: AUY
8.49% Completed
650 total failures
Failed: AUZ
8.50% Completed
651 total failures
Failed: AVA
8.52% Completed
652 total failures
Failed: AVAL
8.53% Completed
653 total failures
Failed: AVAV
8.54% Completed
654 total failures
Failed: AVB
8.55% Completed
655 total failures
Failed: AVC
8.56% Completed
656 total failures
Failed: AVD
8.58% Completed
657 total failures
Failed: AVDL
8.59% Completed
658 total failures
Failed: AVEO
8.60% Completed
659 total failures
F

...

Failed: TRT
90.38% Completed
6974 total failures
Failed: TRTN
90.40% Completed
6975 total failures
Failed: TRTX
90.41% Completed
6976 total failures
Failed: TRU
90.42% Completed
6977 total failures
Failed: TRUP
90.43% Completed
6978 total failures
Accepted: TRV
90.44% Completed
6978 total failures
Failed: TRVG
90.46% Completed
6979 total failures
Failed: TRVN
90.47% Completed
6980 total failures
Failed: TRY
90.48% Completed
6981 total failures
Accepted: TS
90.49% Completed
6981 total failures
Failed: TSBK
90.50% Completed
6982 total failures
Failed: TSC
90.52% Completed
6983 total failures
Failed: TSCAP
90.53% Completed
6984 total failures
Failed: TSCO
90.54% Completed
6985 total failures
Failed: TSE
90.55% Completed
6986 total failures
Failed: TSEM
90.56% Completed
6987 total failures
Failed: TSG
90.58% Completed
6988 total failures
Failed: TSI
90.59% Completed
6989 total failures
Failed: TSL
90.60% Completed
6990 total failures
Failed: TSLA
90.61% Completed
6991 total failures
Failed

Failed: UN
92.38% Completed
7125 total failures
Failed: UNAM
92.39% Completed
7126 total failures
Failed: UNB
92.40% Completed
7127 total failures
Failed: UNF
92.42% Completed
7128 total failures
Failed: UNFI
92.43% Completed
7129 total failures
Accepted: UNH
92.44% Completed
7129 total failures
Failed: UNIT
92.45% Completed
7130 total failures
Failed: UNL
92.46% Completed
7131 total failures
Accepted: UNM
92.48% Completed
7131 total failures
Failed: UNMA
92.49% Completed
7132 total failures
Accepted: UNP
92.50% Completed
7132 total failures
Failed: UNT
92.51% Completed
7133 total failures
Failed: UNTY
92.52% Completed
7134 total failures
Failed: UNV
92.54% Completed
7135 total failures
Failed: UNVR
92.55% Completed
7136 total failures
Failed: UONE
92.56% Completed
7137 total failures
Failed: UONEK
92.57% Completed
7138 total failures
Failed: UOS
92.58% Completed
7139 total failures
Failed: UPD
92.60% Completed
7140 total failures
Failed: UPL
92.61% Completed
7141 total failures
Failed

Failed: VLY
94.38% Completed
7279 total failures
Failed: VLY.WS
94.39% Completed
7280 total failures
Failed: VLY^A
94.40% Completed
7281 total failures
Failed: VLY^B
94.41% Completed
7282 total failures
Failed: VMBS
94.42% Completed
7283 total failures
Failed: VMC
94.44% Completed
7284 total failures
Failed: VMG
94.45% Completed
7285 total failures
Failed: VMI
94.46% Completed
7286 total failures
Failed: VML
94.47% Completed
7287 total failures
Failed: VMO
94.48% Completed
7288 total failures
Failed: VMS
94.50% Completed
7289 total failures
Failed: VMT
94.51% Completed
7290 total failures
Failed: VMW
94.52% Completed
7291 total failures
Failed: VMX
94.53% Completed
7292 total failures
Failed: VMY
94.54% Completed
7293 total failures
Failed: VN8
94.56% Completed
7294 total failures
Failed: VNCE
94.57% Completed
7295 total failures
Failed: VNDA
94.58% Completed
7296 total failures
Failed: VNE
94.59% Completed
7297 total failures
Failed: VNET
94.61% Completed
7298 total failures
Failed: V

Accepted: WETF
96.37% Completed
7436 total failures
Failed: WEX
96.38% Completed
7437 total failures
Failed: WEYS
96.40% Completed
7438 total failures
Failed: WF
96.41% Completed
7439 total failures
Failed: WFB
96.42% Completed
7440 total failures
Accepted: WFC
96.43% Completed
7440 total failures
Failed: WFC.WS
96.44% Completed
7441 total failures
Failed: WFC^J.CL
96.46% Completed
7442 total failures
Failed: WFC^L
96.47% Completed
7443 total failures
Failed: WFC^N
96.48% Completed
7444 total failures
Failed: WFC^O
96.49% Completed
7445 total failures
Failed: WFC^P
96.50% Completed
7446 total failures
Failed: WFC^Q
96.52% Completed
7447 total failures
Failed: WFC^R
96.53% Completed
7448 total failures
Failed: WFC^T
96.54% Completed
7449 total failures
Failed: WFC^V
96.55% Completed
7450 total failures
Failed: WFC^W
96.56% Completed
7451 total failures
Failed: WFC^X
96.58% Completed
7452 total failures
Failed: WFC^Y
96.59% Completed
7453 total failures
Failed: WFE
96.60% Completed
7454 

Failed: WZR
98.37% Completed
7591 total failures
Failed: X
98.38% Completed
7592 total failures
Failed: XAM
98.39% Completed
7593 total failures
Failed: XAN
98.40% Completed
7594 total failures
Failed: XAN^C
98.42% Completed
7595 total failures
Failed: XBIO
98.43% Completed
7596 total failures
Failed: XBIT
98.44% Completed
7597 total failures
Failed: XCL
98.45% Completed
7598 total failures
Failed: XCRA
98.46% Completed
7599 total failures
Failed: XEC
98.48% Completed
7600 total failures
Failed: XEL
98.49% Completed
7601 total failures
Failed: XELA
98.50% Completed
7602 total failures
Failed: XELB
98.51% Completed
7603 total failures
Failed: XENE
98.52% Completed
7604 total failures
Failed: XENT
98.54% Completed
7605 total failures
Failed: XERS
98.55% Completed
7606 total failures
Failed: XF1
98.56% Completed
7607 total failures
Failed: XFLT
98.57% Completed
7608 total failures
Failed: XGTI
98.58% Completed
7609 total failures
Failed: XHR
98.60% Completed
7610 total failures
Failed: XI

In [4]:
filterd_dgi = dgi_df[(dgi_df['debtEquityAvg'] <= 1.5) &
       (dgi_df['divGrowth'] >= 0.03) &
       (dgi_df['payoutChange'] <= 0) &
       (dgi_df['payoutAvg'] <= 0.6) & (dgi_df['payoutAvg'] >= 0) &
       (dgi_df['netMarginAvg'] >= 0.07) &
       (dgi_df['roicAvg'] >= 0.2)].sort_values('divGrowth', ascending = False)

filterd_dgi.to_csv('filtered_dgi_names.csv')

In [ ]:
# lookback_years = 3
# dgi_list = []

# for ticker in earnings_df['Underlying'].drop_duplicates().tolist():
#     curr_earnings = earnings_df[earnings_df['Underlying'] == ticker]

#     dgi_df = pd.DataFrame(index = [ticker])
#     dgi_df['payoutChange'] = curr_earnings.payoutRatio.sort_index().pct_change().tail(lookback_years).mean()
#     dgi_df['divGrowth'] = curr_earnings.dividendsPaid.sort_index().pct_change().tail(lookback_years).mean()
#     dgi_df['payoutAvg'] = curr_earnings.payoutRatio.sort_index().tail(lookback_years).mean()
#     dgi_df['debtEquityAvg'] = curr_earnings.debtEquityRatio.sort_index().tail(lookback_years).mean()
#     dgi_df['netMarginAvg'] = curr_earnings.netMargin.sort_index().tail(lookback_years).mean()
#     dgi_df['roicAvg'] = curr_earnings.roic.sort_index().tail(lookback_years).mean()
#     dgi_list.append(dgi_df)
